#Chatbot_Nut v0.2a



In [2]:
import re #Permite usar expresiones regular
import nltk #Herramientas de lenguaje natural
import numpy as np #Para la estructuras de datos y matrices
import random #Herramienta para generar números pseudoaleatorios
import json #Herramienta para el intercambio de datos y para analizar los datos

import torch #Libreria para el aprendizaje automatico, utiliza los tensorflow, ejecuta el codigo de forma nativa usando la GPU
import torch.nn as nn #Permite construir redes neuronales
from torch.utils.data import Dataset, DataLoader #Permite indicar el conjunto de datos que que se cargará

nltk.download('punkt') #Este tokenizer divide el texto en lista de oraciones.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#Funciones para tokenizar (tokenize), derivar (stem) y bag_of_words (bolsa de palabras) 
from nltk.stem.porter import PorterStemmer #libreria para obtener las derivadas de las palabrasmas frecuentes
stemmer = PorterStemmer()

def tokenize(oraciones):
    """
    dividir la oración en una matriz de palabras / fichas
    un token puede ser una palabra o un carácter de puntuación, o un número
    """
    return nltk.word_tokenize(oraciones)


def stem(palabra): #derivar
    """
    derivar = encontrar la forma raíz de la palabra
    ejemplos:
    palabras = ["comiamos", "comemos", "comías"]
    palabras = [raíz (w) para w en palabras]
    -> ["comer", "comer", "comer"]
    """
    return stemmer.stem(palabra.lower())


def bag_of_words(tokenized_sentence, palabras):
    """
    bolsa de devolución de matriz de palabras:
     1 por cada palabra conocida que existe en la oración, 0 en caso contrario
     ejemplo:
     frase = ["hola", "cómo", "estas", "tú"]
     palabras =                   ["hola", "hola", "yo", "tú", "adiós", "gracias", "genial"]
     palabras de entrenamiento =  [  0   ,    1  ,  0  ,  1  ,    0   ,    0     ,    0    ]
    """
    # derivar cada palabra
    sentence_words = [stem(word) for word in tokenized_sentence]
    # inicializar bolsa con 0 para cada palabra
    bag = np.zeros(len(palabras), dtype=np.float32)
    for idx, w in enumerate(palabras):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [4]:
#función para nuestro modelo de RNN
class NeuralNet(nn.Module):
    def __init__(self, tamanio_de_entrada, tamanio_oculto, numero_de_clases):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(tamanio_de_entrada, tamanio_oculto) 
        self.l2 = nn.Linear(tamanio_oculto, tamanio_oculto) 
        self.l3 = nn.Linear(tamanio_oculto, numero_de_clases)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # sin activación y sin softmax al final
        return out

In [5]:
#cargamos nuestra bd
with open('basededatosNut.json', 'r') as f:
    dietas = json.load(f)

lista_de_palabras = []
etiquetas = []
xy = []
#iterar cada oración de nuestro patron de dietas
for diet in dietas['dietas']:
    etiqueta = diet['etiqueta'] 
    etiquetas.append(etiqueta) #agregar a la lista de etiquetas
    for patron in diet['patrones']:
        # tokenizar cada palabra de la oración
        w = tokenize(patron)
        # agregar a nuestra lista de palabras
        lista_de_palabras.extend(w)
        # agregar al par xy
        xy.append((w, etiqueta))

# ignorar palabra
ignorar_signos = ['?', '.', '!']
#convierte minuscula e ignora signos
lista_de_palabras = [stem(w) for w in lista_de_palabras if w not in ignorar_signos]
# eliminar duplicados y ordenar
lista_de_palabras = sorted(set(lista_de_palabras))
etiquetas = sorted(set(etiquetas))

print(len(xy), "patrones")
print(len(etiquetas), "etiquetas:", etiquetas)
print(len(lista_de_palabras), "palabras derivadas unicas:", lista_de_palabras)

# crear datos de entrenamiento
X_cola = []
Y_cola = []
for (patron_de_oracion, etiqueta) in xy:
    # X: bolsa de palabras para cada patron de sentencia
    bolsa = bag_of_words(patron_de_oracion, lista_de_palabras)
    X_cola.append(bolsa)
    # y: PyTorch CrossEntropyLoss solo necesita etiquetas de clase, no one-hot
    ETIQUETA_ = etiquetas.index(etiqueta)
    Y_cola.append(ETIQUETA_)

X_cola = np.array(X_cola)
Y_cola = np.array(Y_cola)

# Hiperparámetros para RNN
interaciones = 1000 #NUMERO DE INTERACIONES
batch_size = 8      #tamanio de lote
tasa_de_aprendizaje = 0.001
tamanio_de_entrada = len(X_cola[0])
tamanio_oculto = 8
tamanio_de_salida = len(etiquetas)
print(tamanio_de_entrada, tamanio_de_salida)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_cola)
        self.x_data = X_cola
        self.y_data = Y_cola

    # Admite la indexación de modo que el conjunto de datos [i] se pueda utilizar para obtener la i-ésima muestra
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # podemos llamar a len (conjunto de datos) para devolver el tamaño
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
cargador_de_cola = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

dispositivo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelo = NeuralNet(tamanio_de_entrada, tamanio_oculto, tamanio_de_salida).to(dispositivo)

# Pérdida y optimizacion
criterio = nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(modelo.parameters(), lr=tasa_de_aprendizaje)

# Entrena el modelo
for nivel in range(interaciones):
    for (PALABRAS, ETIQUETAS) in cargador_de_cola:
        PALABRAS = PALABRAS.to(dispositivo)
        ETIQUETAS = ETIQUETAS.to(dtype=torch.long).to(dispositivo)
        
        # Pase adelantado
        SALIDAS = modelo(PALABRAS)
        # si y sería one-hot, debemos aplicar
        # etiquetas = antorcha.max (etiquetas, 1) [1]
        PERDIDAS = criterio(SALIDAS, ETIQUETAS)
        
        # Retroceder y optimizar
        optimizador.zero_grad()
        PERDIDAS.backward()
        optimizador.step()
        
    if (nivel+1) % 100 == 0:
        print (f'interaciones [{nivel+1}/{interaciones}], perdidos: {PERDIDAS.item():.4f}')


print(f'perdida final: {PERDIDAS.item():.4f}')

datos = {
"estado_del_modelo": modelo.state_dict(),
"tamanio_de_entrada": tamanio_de_entrada,
"tamanio_oculto": tamanio_oculto,
"tamanio_de_salida": tamanio_de_salida,
"lista_de_palabras": lista_de_palabras,
"etiquetas": etiquetas
}

FILE = "datos.pth"
torch.save(datos, FILE)

print(f'Entrenamiento completado, archivo guardado en: {FILE}')

88 patrones
16 etiquetas: ['Comer', 'Dietasadulto', 'Dietasniño', 'IMC', 'TiposdeFrutas', 'Tiposdealimentos', 'adios', 'dietaenfermedades', 'dietas', 'edad', 'estado', 'gracias', 'peso', 'saludo', 'talla', 'tipo']
107 palabras derivadas unicas: ['1,80', '1,80m', '1.20', '1.50', '1.75', '1.80', '1.80m', '10', '20', '30', '50kg', '60', '60kg', '65k', '68k', '68kg', '70', '70k', '70kg', 'adió', 'adulto', 'ahí', 'algo', 'alguien', 'alrededor', 'año', 'bajar', 'bien', 'buen', 'bye', 'calculo', 'cansado', 'cardiovascular', 'clasificacion', 'cocinar', 'coma', 'comer', 'crecer', 'cómo', 'dame', 'de', 'deportista', 'dieta', 'dime', 'dulc', 'día', 'el', 'enfermedad', 'es', 'eso', 'estoy', 'está', 'evitar', 'excelent', 'favor', 'fruta', 'gana', 'gracia', 'hambr', 'hasta', 'hello', 'hola', 'holi', 'hoy', 'imc', 'kg', 'luego', 'm', 'mal', 'mayor', 'mi', 'mido', 'mucha', 'muy', 'neutra', 'niño', 'no', 'nut', 'oye', 'para', 'pero', 'peso', 'por', 'problema', 'puedo', 'que', 'quiero', 'resfrio', 'rico

In [6]:
#Declaramos el objeto torch
dispositivo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Cargamos nuestra bd
with open('basededatosNut.json', 'r') as json_data:
    DIETAS = json.load(json_data)

#Cargamos los datos de entrenamiento
FILE = "datos.pth"
datos = torch.load(FILE)

#Obtenemos los parámetros
_tamanio_entrada = datos["tamanio_de_entrada"]
_tamanio_oculto = datos["tamanio_oculto"]
_tamanio_salida = datos["tamanio_de_salida"]
_lista_de_palabras = datos['lista_de_palabras']
_etiqueta = datos['etiquetas']
_estado_modelo = datos["estado_del_modelo"]

#Crea el modelo de reconocimiento
MODELO = NeuralNet(_tamanio_entrada, _tamanio_oculto, _tamanio_salida).to(dispositivo)
MODELO.load_state_dict(_estado_modelo)
MODELO.eval() #evalua las respuestas


bot_name = "BotNut"
print(f"{bot_name}: Mi nombre es BotNut. Responderé a tus consultas, si desea salir, escriba adios")
while True:
    sentencias = input("Tu: ")
    if sentencias == "adios":
        print(f"{bot_name}:Que tengas un excelente día")
        break

    sentencias = tokenize(sentencias) #Tokenizamos las palabras
    X = bag_of_words(sentencias, _lista_de_palabras) #Coincidencia de palabras con la bd json
    X = X.reshape(1, X.shape[0]) 
    X = torch.from_numpy(X).to(dispositivo) #Objeto de entrenamiento y posibles respuestas

    salida = MODELO(X) 
    _, prediccion = torch.max(salida, dim=1)

    ETIQUETA = _etiqueta[prediccion.item()] #Etiquetas y predicciones

    problemas = torch.softmax(salida, dim=1) #Entrada de n dimensiones y rescalado de salidas de n dimensiones
    probabilidad = problemas[0][prediccion.item()] #Prediccion de la respuesta mas acertada
    if probabilidad.item() > 0.75: #Probabilidades mayores a 75%
        for diets in DIETAS['dietas']: #Posibles dietas
            if ETIQUETA == diets["etiqueta"]:
                print(f"{bot_name}: {random.choice(diets['respuestas'])}")
    else:
        print(f"{bot_name}: No entiendo...")

BotNut: Mi nombre es BotNut. Responderé a tus consultas, si desea salir, escriba adios
Tu: hola
BotNut: Hello I'm Nut, espero tengas un buen dia
Tu: adios
BotNut:Que tengas un excelente día
